In [1]:
from occultence import *

In [23]:
targ = LightCurve(name = "test",
                  time = np.linspace(10,11,10),
                  flux = np.arange(10)+100,
                  uncertainty = np.ones(10))

🌈🤖 Warning! The time array is not an astropy.Time object, therefore there is no info about the 
format or scale.
We will assume that it is JD and TDB from here on!



In [24]:
targ.__dict__

{'_core_dictionaries': ['timelike', 'metadata'],
 'metadata': {'name': 'test'},
 'timelike': {'time': <Time object: scale='tdb' format='jd' value=[10.         10.11111111 10.22222222 10.33333333 10.44444444 10.55555556
   10.66666667 10.77777778 10.88888889 11.        ]>,
  'flux': array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109]),
  'uncertainty': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}}

In [25]:
clean_targ = targ.clean(mask_dust=False, mask_bad_weather=True, mask_cosmics=True)

AttributeError: 'LightCurve' object has no attribute 'clean'

In [8]:
bin_targ = clean_targ.bin(dt=5*u.minute)
gp_targ = bin_targ.gp_detrend(first_sigma_clip=True, second_sigma_clip=True, sigma=3, ...)
bls_targ = gp_targ.bls_search(min_period=0.5, max_period=100, transit_duration=[0.1, 0.2, 0.3], ...)```

{'_core_dictionaries': ['timelike', 'metadata'],
 'metadata': {'name': 'test'},
 'timelike': {'time': <Time object: scale='tdb' format='jd' value=[10.         10.11111111 10.22222222 10.33333333 10.44444444 10.55555556
   10.66666667 10.77777778 10.88888889 11.        ]>,
  'flux': array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109]),
  'uncertainty': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])}}